Import Libraries

In [ ]:
import pandas as pd
import time
import os

Data Preparation

In [ ]:
root_dir = 'source/used'
files = os.listdir(root_dir)
all_dfs = []

In [ ]:
for file in files:
    df = pd.read_csv(root_dir + '/' + file)
    df.iloc[0,-1] = 'Team'
    new_columns = [f"{col}_{df.iloc[0][col]}" for col in df.columns]
    df = df[1:]
    df.columns = new_columns
    all_dfs.append(df)

df = pd.concat(all_dfs,ignore_index = True)

In [ ]:
df = df.dropna()
df
df[df['Unnamed: 0_level_0_Player'] == 'Bruno Fernandes']


,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_#,Unnamed: 2_level_0_Nation,Unnamed: 3_level_0_Pos,Unnamed: 4_level_0_Age,Unnamed: 5_level_0_Min,Performance_Gls,Performance.1_Ast,Performance.2_PK,Performance.3_PKatt,...,SCA.1_GCA,Passes_Cmp,Passes.1_Att,Passes.2_Cmp%,Passes.3_PrgP,Carries_Carries,Carries.1_PrgC,Take-Ons_Att,Take-Ons.1_Succ,Team_Team
2272,Bruno Fernandes,18.0,pt POR,"AM,DM",27-227,83,0,0,0,1,...,0,26,36,72.2,3,23,1,1,0,Manchester United
2334,Bruno Fernandes,8.0,pt POR,CM,30-087,90,0,0,0,0,...,0,59,68,86.8,6,51,3,1,0,Manchester United
2364,Bruno Fernandes,8.0,pt POR,"AM,RW",28-136,90,0,1,0,0,...,1,30,40,75.0,3,19,1,0,0,Manchester United
2393,Bruno Fernandes,18.0,pt POR,AM,26-144,90,0,0,0,0,...,0,43,53,81.1,7,28,2,0,0,Manchester United
2454,Bruno Fernandes,8.0,pt POR,AM,28-360,90,0,0,0,0,...,0,31,43,72.1,4,27,1,0,0,Manchester United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75974,Bruno Fernandes,18.0,pt POR,AM,27-011,90,0,0,0,0,...,0,59,80,73.8,6,50,3,2,1,Manchester United
79602,Bruno Fernandes,18.0,pt POR,AM,26-355,90,0,0,0,0,...,0,37,50,74.0,7,35,3,0,0,Manchester United
79630,Bruno Fernandes,8.0,pt POR,AM,30-109,46,0,0,0,0,...,0,24,31,77.4,4,26,0,3,1,Manchester United
79668,Bruno Fernandes,8.0,pt POR,AM,28-114,89,0,1,0,0,...,1,44,54,81.5,13,42,0,1,0,Manchester United
